In [1]:
# Custom Imports
from polysaccharide import analysis, filetree, general, logutils, molutils
from polysaccharide import simulation as polysim
from polysaccharide import representation as rep

from polysaccharide import charging
from polysaccharide.charging.types import AtomIDMap, ResidueChargeMap
from polysaccharide.charging.residues import ChargedResidue
from polysaccharide.charging.application import CHARGER_REGISTRY, ChargingParameters

from polysaccharide import LOGGERS_MASTER
from polysaccharide.logutils import ProcessLogHandler
from polysaccharide.representation import Polymer, PolymerManager
from polysaccharide.solvation.solvents import WATER_TIP3P
from polysaccharide.analysis import plotprops, trajectory
from polysaccharide.simulation import SimulationPaths, SimulationParameters
from polysaccharide.graphics import rdkdraw

# General Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# File I/O
from pathlib import Path
import csv, json, pickle
from shutil import copyfile, rmtree

# Logging and Shell
import logging
logging.basicConfig(level=logging.INFO)
                            
# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdmolfiles

# Molecular Dynamics
from openff.interchange import Interchange
from openff.toolkit import ForceField
from openff.toolkit.topology import Topology
from openff.toolkit.topology.molecule import Molecule, Atom
from openff.toolkit.typing.engines.smirnoff.parameters import LibraryChargeHandler

from openff.units import unit
from openmm.unit import picosecond, femtosecond, nanosecond # time
from openmm.unit import nanometer, angstrom # length
from openmm.unit import kelvin, atmosphere # misc

# Static Paths
RESOURCE_PATH = Path('resources')
COLL_PATH = Path('Collections')
# COMPAT_PDB_PATH = Path('compatible_pdbs')
COMPAT_PDB_PATH = Path('compatible_pdbs_updated')

/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
<f

In [ ]:
PolymerManager

## Configuring and (re)loading polymers, setting solvents, checking validity

In [2]:
reset      = False #True
purge_sims = False #True 
purge_logs = False #True

# poly_source_path = COMPAT_PDB_PATH / 'simple_polymers'
poly_source_path = COMPAT_PDB_PATH / 'water_soluble_small'
# poly_source_path = COMPAT_PDB_PATH / 'water_soluble_large'
solv_template    = RESOURCE_PATH/'inp_templates'/'solv_polymer_template_box.inp'
desired_solvents = (WATER_TIP3P,) # (None,)
exclusion = 1.0*nanometer

# Define derived paths and create manager
collection_path  = COLL_PATH / poly_source_path.name
structure_path   = poly_source_path / f'{poly_source_path.name}_structures'
monomer_path     = poly_source_path / f'{poly_source_path.name}_monomers'

mgr = PolymerManager(collection_path)

# Perform manager setup / purge actions
if purge_logs: # NOTE : must be done BEFORE log FileHandler is created, as this will destroy it's output as well
    mgr.purge_logs(really=True)

creation_logger = logging.getLogger('polymer_setup')
loggers = [creation_logger, *LOGGERS_MASTER]

with logutils.ProcessLogHandler(filedir=mgr.log_dir, loggers=loggers, proc_name=f'Setup of {mgr.collection_dir.name}', timestamp=True):
    if reset:
        mgr.purge_collection(really=True, purge_logs=False) # Explicitly DON'T purge logs here (will be done prior to entering log loop)

    if purge_sims:
        mgr.purge_sims(really=True)

    if not mgr.polymers: # will be empty if not yet instantiated or if reset prior
        mgr.populate_collection(struct_dir=structure_path, monomer_dir=monomer_path)
        mgr.solvate_collection(desired_solvents, template_path=solv_template, exclusion=exclusion)

INFO:polysaccharide.representation:Built filetree for paam_modified
INFO:polysaccharide.representation:Acquiring structure file(s) paam_modified.pdb from compatible_pdbs_updated/water_soluble_small/water_soluble_small_structures
INFO:polysaccharide.representation:Acquiring monomer file(s) paam_modified.json from compatible_pdbs_updated/water_soluble_small/water_soluble_small_monomers
INFO:polysaccharide.representation:
INFO:polysaccharide.representation:Built filetree for peg_modified
INFO:polysaccharide.representation:Acquiring structure file(s) peg_modified.pdb from compatible_pdbs_updated/water_soluble_small/water_soluble_small_structures
INFO:polysaccharide.representation:Acquiring monomer file(s) peg_modified.json from compatible_pdbs_updated/water_soluble_small/water_soluble_small_monomers
INFO:polysaccharide.representation:
INFO:polysaccharide.representation:Built filetree for pnipam_modified
INFO:polysaccharide.representation:Acquiring structure file(s) pnipam_modified.pdb from

In [3]:
# Selecting subset of molecules which is suitable for ABE10 charging and subsequent simulation
HARD_POLYMERS = ['vulcanizedrubber', 'polyphenylenesulfone', 'polyethylene', 'polyphenyleneI'] # pathological or otherwise difficult-to-run polymers that I've encountered
EASY_POLYMERS = ['polyvinylchloride_solv_water', 'polyethylmethacrylate_solv_water'] + ['naturalrubber_solv_water', 'PEO_PLGA_solv_water'] # the quickest-to-charge and -to-simulate polymers in the simple_polymers sample set

whitelisted  = rep.filter_factory_by_attr(attr_name='base_mol_name', condition=lambda name : name in HARD_POLYMERS, inclusive=False)
good_solvent = rep.filter_factory_by_attr(attr_name='solvent', condition=lambda solv : solv in desired_solvents)

filters : tuple[rep.MolFilter] = (whitelisted, good_solvent, rep.has_monomers, rep.AM1_sized)
# filters : tuple[rep.MolFilter] = (whitelisted, good_solvent, rep.has_monomers) # for large molecules
valid_mols = mgr.filtered_by(filters)

# display to check that loading has gone as planned
print(valid_mols)

{'paam_modified_solv_water': Polymer(mol_name=paam_modified_solv_water, base_mol_name=paam_modified, path=Collections/water_soluble_small/paam_modified/paam_modified_solv_water, exclusion=1.0 nm, solvent=Solvent(name='water', formula='H2O', smarts='[#1:1]-[#8:3]-[#1:2]', density=Quantity(value=0.997, unit=gram/(centimeter**3)), MW=Quantity(value=18.015, unit=gram/mole), charges={'1': 0.417, '2': 0.417, '3': -0.834}, structure_file=PosixPath('/home/timber/Documents/Python/openff-workspace/polymer_workspace/polysaccharide/solvation/solvents/WATER_TIP3P/water.pdb'), forcefield_file=PosixPath('/home/timber/Documents/Python/openff-workspace/polymer_workspace/polysaccharide/solvation/solvents/WATER_TIP3P/tip3p.offxml'))), 'pnipam_modified_solv_water': Polymer(mol_name=pnipam_modified_solv_water, base_mol_name=pnipam_modified, path=Collections/water_soluble_small/pnipam_modified/pnipam_modified_solv_water, exclusion=1.0 nm, solvent=Solvent(name='water', formula='H2O', smarts='[#1:1]-[#8:3]-[#

## Charge and sim loop V2

In [5]:
# DEFINE TARGET MOLECULES AND FORCEFIELD
# sample_mols = EASY_POLYMERS
# sample_mols = ['polyvinylchloride_solv_water']#, 'naturalrubber_solv_water']
sample_mols = valid_mols

# CHARGING PARAMETERS
perform_charging = True
chg_params_path = RESOURCE_PATH / 'chg_templates' / 'standard_chg_params.json'
# chg_params_path = RESOURCE_PATH / 'chg_templates' / 'long_chain_chg_params.json'
# chg_params_path = RESOURCE_PATH / 'chg_templates' / 'standard_chg_params_rewrite.json'
chg_params = ChargingParameters.from_file(chg_params_path)

# SIMULATION PARAMETERS
run_sims = True
sim_param_path = RESOURCE_PATH / 'sim_templates' / 'debug_sim.json'
# sim_param_path = RESOURCE_PATH / 'sim_templates' / 'half_standard_sim.json'
# sim_param_path = RESOURCE_PATH / 'sim_templates' / 'standard_sim.json'
sim_params = SimulationParameters.from_file(sim_param_path)

# ANALYSIS PARAMETERS
perform_analysis = False #True
traj_sample_interval = 5 # interval between consecutive time points for trajectory analysis (needed for longer sims)


# PRE-FLIGHT CALCULATIONS
if chg_params.averaging_charge_method == 'ABE10_averaged':
    raise ValueError('Charge averaging must be performed over a non-averaged (i.e. pure) set of charges')

sample_dirs = {
    mol_name : mgr.polymers[mol_name] # TOSELF : deliberately not using .get() so this raises an easier-to-debug KeyError
        for mol_name in sample_mols
}

step_names = {
    'charging' : perform_charging,
    'simulation' : run_sims,
    'analysis' : perform_analysis
}
action_str = ' & '.join([
    step_name
        for step_name, perform_step in step_names.items()
            if perform_step
])

print(chg_params)
print(sim_params)
print(sim_params.num_steps, sim_params.record_freq, action_str)
print(sample_dirs.keys())

ChargingParameters(overwrite_ff_xml=False, overwrite_chg_mono=False, base_ff_path=PosixPath('resources/force_fields/openff_constrained-2.0.0.offxml'), charge_methods=['ABE10_exact', 'Espaloma_AM1BCC', 'ABE10_averaged'], averaging_charge_method='ABE10_exact')
SimulationParameters(total_time=Quantity(value=1, unit=picosecond), num_samples=500, charge_method='ABE10_exact', reported_state_data={'step': True, 'time': True, 'potentialEnergy': True, 'kineticEnergy': True, 'totalEnergy': True, 'temperature': True, 'volume': True, 'density': True, 'progress': False, 'remainingTime': False, 'speed': True, 'elapsedTime': True}, timestep=Quantity(value=1, unit=femtosecond), temperature=Quantity(value=300, unit=kelvin), pressure=Quantity(value=1, unit=atmosphere), friction_coeff=Quantity(value=1, unit=/picosecond), barostat_freq=1)
1000 2 charging & simulation
dict_keys(['paam_modified_solv_water', 'pnipam_modified_solv_water', 'peg_modified_solv_water'])


In [6]:
# Functions for performing primary charging, sim, and analysis actions
def obtain_partial_charges(polymer : Polymer, main_logger : logging.Logger, chg_params : ChargingParameters) -> None:
    '''Ensure a Polymer has all partial charge sets'''
    # 0) LOAD MOLECULE AND TOPOLOGY, ATTEMPT TO APPLY LIBRARY CHARGES
    if not polymer.has_monomer_data:
        raise FileExistsError(f'No monomer JSONs found for {polymer.mol_name}')

    # 1) ENSURING CHARGES AND RELATED FILES FOR ALL CHARGING METHODS EXIST
    for chg_method in chg_params.charge_methods:
        chgr = CHARGER_REGISTRY[chg_method]()
        if chg_method == 'ABE10_averaged': # !NOTE! - critical that this not be the first key in the registry (has nothing to average over from scratch)
            residue_charges = polymer.residue_charges(
                averaging_charge_method=chg_params.averaging_charge_method,
                overwrite_charged_monomer_file=chg_params.overwrite_chg_mono
            )
            chgr.set_residue_charges(residue_charges)
        polymer.assert_charges_for(chgr, return_cmol=False)

    if (polymer.ff_file is None) or chg_params.overwrite_ff_xml: # can only reach if a charged monomer json already exists
        main_logger.info('Acquiring Force Field file with Library Charges')
        forcefield, lib_chgs = polymer.create_FF_file(xml_src=chg_params.base_ff_path, return_lib_chgs=True)

def perform_prop_analysis(polymer : Polymer, main_logger : logging.Logger, traj_sample_interval : int, plot : bool=False):
    '''Analyze trajectories to obtain polymer property data'''
    # aqcuire files for all information
    sim_folder = polymer.newest_sim_dir
    main_logger.info(f'Acquiring trajectory data from {sim_folder}')
    sim_paths = SimulationPaths.from_file(next(sim_folder.glob('*_paths.json')))

    sim_params = SimulationParameters.from_file(sim_paths.sim_params)
    state_data = pd.read_csv(sim_paths.state_data)
    traj = trajectory.load_traj(sim_paths.trajectory, topo_path=polymer.structure_file, sample_interval=traj_sample_interval, remove_solvent=True)

    # save and plot RDF data
    main_logger.info('Determining pairwise radial distribution functions')
    rdf_dataframe = trajectory.acquire_rdfs(traj, max_rad=1.0*nanometer)
    rdf_dataframe.to_csv(sim_folder/'rdfs.csv')
    if plot:
        rdf_fig, rdf_ax = plotprops.plot_rdfs(rdf_dataframe, scale=15.0)
        rdf_fig.suptitle(f'Pairwise Radial Distribution Functions - {polymer.mol_name}')
        rdf_fig.savefig(sim_folder/f'RDFs.png', bbox_inches='tight')
    plt.close()

    # save and plot property data
    main_logger.info('Determining polymer shape properties')
    prop_dataframe = trajectory.acquire_time_props(traj, properties=analysis.polyprops.DEFAULT_PROPS, time_points=sim_params.time_points[::traj_sample_interval]) 
    prop_dataframe.to_csv(sim_folder/'time_series.csv')
    if plot:
        prop_fig, prop_ax = plotprops.plot_time_props(prop_dataframe, scale=18.0)
        prop_fig.suptitle(f'Polymer Shape Properties - {polymer.mol_name}')
        prop_fig.savefig(sim_folder/f'shape_props.png', bbox_inches='tight')
    plt.close()
    
    main_logger.info('Successfully exported trajectory analysis data and plots')

In [7]:
# BEGIN CHARGING / SIM LOOP - Perform charge averaging on all target molecules which don't already have averaged LCs; Load forcefield for those which already do 
main_logger = logging.getLogger(__name__)
loggers = [main_logger, *LOGGERS_MASTER] # loggers from all modules which produce logging output

with logutils.ProcessLogHandler(filedir=mgr.log_dir, loggers=loggers, proc_name=action_str.capitalize(), timestamp=True) as msf_handler:
    for i, (mol_name, polymer) in enumerate(sample_dirs.items()):
        main_logger.info(f'Current molecule: "{mol_name}" ({i + 1}/{len(sample_dirs)})') # +1 converts to more human-readable 1-index for step count
        with msf_handler.subhandler(filedir=polymer.logs, loggers=loggers, proc_name=action_str.capitalize(), timestamp=True) as subhandler: # also log actions to individual Polymers
            if perform_charging:
                main_logger.info('(1) Acquiring partial charges')
                obtain_partial_charges(polymer, main_logger, chg_params)
                main_logger.info('') # log empty line to give some breathing room

            if run_sims:
                main_logger.info('(2) Preparing simulation')
                sim_folder = polymer.run_simulation_NPT(sim_params)
                main_logger.info('') # log empty line to give some breathing room
            
            if perform_analysis:
                main_logger.info('(3) Analyzing trajectories')
                perform_prop_analysis(polymer, main_logger, traj_sample_interval)
                main_logger.info('') # log empty line to give some breathing room

INFO:__main__:Current molecule: "paam_modified_solv_water" (1/3)
INFO:__main__:(1) Acquiring partial charges
INFO:polysaccharide.representation:Loading OpenFF Topology WITH monomer graph match
INFO:polysaccharide.charging.application:Assigning partial charges via the "ABE10_exact" method
/home/timber/Documents/Python/openff-workspace/openff-toolkit/openff/toolkit/topology/molecule.py:2493: UserWarning: Warning! Partial charge method 'am1bccelf10' is not designed for use on large (i.e. > 150 atoms) molecules and may crash or take hours to run on this molecule (found 172 atoms). For more, see https://docs.openforcefield.org/projects/toolkit/en/stable/faq.html#parameterizing-my-system-which-contains-a-large-molecule-is-taking-forever-whats-wrong
  warnings.warn(
